In [17]:
def generate(x):
    return x
    
from IPython.display import HTML
HTML("./main.html")
#kernel.execute(command, {'iopub': {"output": callback}}, {silent:false});

In [5]:
out

NameError: name 'out' is not defined